In [1]:
import invisible_cities.reset.utils as rst_util
import numpy as np
from invisible_cities.core.system_of_units import pes, mm, mus, ns
import invisible_cities.reco.corrections    as corrf
import invisible_cities.database.load_db as dbf 
import reset_functions_event as rstf

# Initialize RESET
iterations = 100
pitch      = 10
run_number = 4495
nsipms     = 1792
npmts      = 1 
dist       = 20. 
sipm_dist  = 20. 
pmt_dist   = 200 # max distance included in the param file
sipm_thr   = 5.
#x_size     = 2.
#y_size     = 2.

x_size     = 10.
y_size     = 10.

rmax       = 198 #
slice_width = 2.
pmt_param  = "/home/jmbenlloch/reset_data/mapas/PMT_Map_corr_keV.h5"
sipm_param = "/home/jmbenlloch/reset_data/mapas/SiPM_Map_corr_z0.0_keV.h5"

pmap_file = '4495_21215.h5'

pmap_conf = {}
pmap_conf['s1_emin'] = 54 * pes 
pmap_conf['s1_emax'] = 2400. * pes 
pmap_conf['s1_wmin'] = 7*25 * ns
pmap_conf['s1_wmax'] = 16*25 * ns
pmap_conf['s1_hmin'] = 12. * pes 
pmap_conf['s1_hmax'] = 400. * pes 
pmap_conf['s1_num'] = 1 
pmap_conf['s1_ethr'] = 0.5 * pes 

pmap_conf['s2_emin'] = 200000. * pes 
pmap_conf['s2_emax'] = 2.e6 * pes 
pmap_conf['s2_wmin'] = 4.* mus 
pmap_conf['s2_wmax'] = 500. * mus 
pmap_conf['s2_hmin'] = 600. * pes 
pmap_conf['s2_hmax'] = 60000. * pes 
pmap_conf['s2_numMin'] = 1 
pmap_conf['s2_numMax'] = 3 
pmap_conf['s2_ethr'] = 0. * pes 
pmap_conf['s2_nsipmmin'] = 1 
pmap_conf['s2_nsipmmax'] = 1792

# Read file and select peaks
selector = rst_util.refresh_selector(pmap_conf)
s1s, s2s, s2sis, peaks = rst_util.load_and_select_peaks(pmap_file, 21215, selector)

# Lifetime correction
ZCorr = corrf.LifetimeCorrection(1093.77, 23.99)

# Sensors info
data_sipm = dbf.DataSiPM(run_number)

# Take the first peak
peak = next(iter(peaks))
evt = 21215
sipm_thr = sipm_thr

# Prepare data
voxels_data, slices, energies, zs = rst_util.prepare_data(s1s, s2s, s2sis, slice_width, evt, peak, data_sipm, nsipms, sipm_thr, dist, ZCorr)
slices_start = rst_util.slices_start(voxels_data, x_size, y_size)

In [2]:
import invisible_cities.reset.reset_serial as rst_serial

# Create voxels

In [3]:
rst_voxels = rst_serial.create_voxels(voxels_data, slices_start, x_size, y_size, rmax)

In [4]:
rst_voxels

ResetVoxels(nslices=6, nvoxels=1717, voxels=array([(115., 25.,  6.3964443), (115., 35.,  6.3964443),
       (115., 45.,  6.3964443), ..., (195.,  5., 10.671945 ),
       (195., 15., 10.671945 ), (195., 25., 10.671945 )],
      dtype=[('x', '<f4'), ('y', '<f4'), ('E', '<f4')]), slice_ids=array([0, 0, 0, ..., 5, 5, 5], dtype=int32), slice_start=array([   0,   67,  173,  470,  872, 1404, 1717], dtype=int32), address=array([   0,    1,    2, ..., 1717, 1717, 1717]))

# Anode response

In [5]:
nslices = voxels_data.xmin.shape[0]
anode_response = rst_serial.create_anode_response(slices)

# Create cathode response

In [6]:
cath_response = energies

# Compute probs

In [7]:
import math

In [8]:
sipm_param_file  = "/home/jmbenlloch/reset_data/mapas/SiPM_Map_corr_z0.0_keV.h5"
sipm_params = rst_util.read_corrections_file(sipm_param_file, 'SiPM')

sipms_per_voxel = int(math.floor(2 * sipm_dist / pitch) + 1)**2
voxels_per_sipm = int((2 * sipm_dist)**2 / (x_size * y_size))
xs = data_sipm.X
ys = data_sipm.Y

In [9]:
sipm_probs = rst_serial.compute_probabilities_cuda(rst_voxels, xs, ys, nsipms, sipms_per_voxel, sipm_dist, sipm_params, anode_response)

In [10]:
sipm_probs.nprobs

42853

### PMTs

In [11]:
pmt_param_file = "/home/jmbenlloch/reset_data/mapas/PMT_Map_corr_keV.h5"
pmt_params = rst_util.read_corrections_file(pmt_param_file, 'PMT')

pmts_per_voxel = npmts
voxels_per_pmt = int((2 * pmt_dist)**2 / (x_size * y_size))

xs_pmts = np.array([0.], dtype='f4')
ys_pmts = np.array([0.], dtype='f4')

In [12]:
pmt_probs = rst_serial.compute_probabilities_cuda(rst_voxels, xs_pmts, ys_pmts, npmts, pmts_per_voxel, pmt_dist, pmt_params, cath_response)

# Sensor probs

In [13]:
sipm_sns_probs = rst_serial.compute_sensor_probs(sipm_probs, nslices, nsipms, rst_voxels.slice_ids)
pmt_sns_probs = rst_serial.compute_sensor_probs(pmt_probs, nslices, npmts, rst_voxels.slice_ids)

# Forward denom

In [14]:
sipm_fwd_denom = rst_serial.forward_denoms(nsipms, nslices, rst_voxels.voxels, sipm_sns_probs)
pmt_fwd_denom = rst_serial.forward_denoms(npmts, nslices, rst_voxels.voxels, pmt_sns_probs)

# MLEM step

In [15]:
rst_serial.compute_mlem(iterations, rst_voxels, nsipms, sipm_probs, sipm_sns_probs, npmts, pmt_probs, pmt_sns_probs)

In [ ]:
rst_voxels

ResetVoxels(nslices=6, nvoxels=1717, voxels=array([(115., 25., 1.1578470e-05), (115., 35., 6.0620841e-02),
       (115., 45., 2.5277985e-05), ..., (195.,  5., 2.4371329e-04),
       (195., 15., 2.5492685e-04), (195., 25., 2.0281288e-04)],
      dtype=[('x', '<f4'), ('y', '<f4'), ('E', '<f4')]), slice_ids=array([0, 0, 0, ..., 5, 5, 5], dtype=int32), slice_start=array([   0,   67,  173,  470,  872, 1404, 1717], dtype=int32), address=array([   0,    1,    2, ..., 1717, 1717, 1717]))

# RESET serial

In [ ]:
rst_voxels = rst_serial.create_voxels(voxels_data, slices_start, x_size, y_size, rmax)
anode_response = rst_serial.create_anode_response(slices)
cath_response = energies

sipm_param_file  = "/home/jmbenlloch/reset_data/mapas/SiPM_Map_corr_z0.0_keV.h5"
sipm_params = rst_util.read_corrections_file(sipm_param_file, 'SiPM')

sipms_per_voxel = int(math.floor(2 * sipm_dist / pitch) + 1)**2
voxels_per_sipm = int((2 * sipm_dist)**2 / (x_size * y_size))
xs = data_sipm.X
ys = data_sipm.Y

sipm_probs = rst_serial.compute_probabilities_cuda(rst_voxels, xs, ys, nsipms, sipms_per_voxel, sipm_dist, sipm_params, anode_response)

pmt_param_file = "/home/jmbenlloch/reset_data/mapas/PMT_Map_corr_keV.h5"
pmt_params = rst_util.read_corrections_file(pmt_param_file, 'PMT')

pmts_per_voxel = npmts
voxels_per_pmt = int((2 * pmt_dist)**2 / (x_size * y_size))

xs_pmts = np.array([0.], dtype='f4')
ys_pmts = np.array([0.], dtype='f4')

pmt_probs = rst_serial.compute_probabilities_cuda(rst_voxels, xs_pmts, ys_pmts, npmts, pmts_per_voxel, pmt_dist, pmt_params, cath_response)

sipm_sns_probs = rst_serial.compute_sensor_probs(sipm_probs, nslices, nsipms, rst_voxels.slice_ids)
pmt_sns_probs = rst_serial.compute_sensor_probs(pmt_probs, nslices, npmts, rst_voxels.slice_ids)

sipm_fwd_denom = rst_serial.forward_denoms(nsipms, nslices, rst_voxels.voxels, sipm_sns_probs)
pmt_fwd_denom = rst_serial.forward_denoms(npmts, nslices, rst_voxels.voxels, pmt_sns_probs)

rst_serial.compute_mlem(iterations, rst_voxels, nsipms, sipm_probs, sipm_sns_probs, npmts, pmt_probs, pmt_sns_probs)

In [ ]:
slices_start

In [ ]:
slices